Import packages

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
%matplotlib inline

Helper functions

In [ ]:
def color_treshold(image):
    upper = np.uint8([255, 255, 255])
    
    # white color mask
    lower = np.uint8([200, 200, 200])
    white_mask = cv2.inRange(image, lower, upper)
    
    # yellow color mask
    lower = np.uint8([190, 190,   0])
    yellow_mask = cv2.inRange(image, lower, upper)
    
    # combine the mask
    combined_mask = cv2.bitwise_or(white_mask, yellow_mask)
    masked = cv2.bitwise_and(image, image, mask = combined_mask)
    return masked

def grayscale(img):
    #Grayscales the image, returns image with one color channel
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
def canny(img, low_threshold, high_threshold):
    #Applies Canny Edge transform
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    #Applies Gaussian blur to the image
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    #Applies image mask, only keeps region of the image defined by mask polygon
    
    #defining a blank mask
    mask = np.zeros_like(img)   
   
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, 255)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=2):
    #drawns lines on the image
    for i in range(0,len(lines)):
        x1,y1,x2,y2 = lines[i]
        cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def draw_lanes(image, lines):
    #separeats lines list into right and left lane
    rightLines, leftLines = separate_lines_by_slope(lines, image.shape[1]/2)
    rightLanePoints = get_lane_points(rightLines, image.shape[0])
    leftLanePoints = get_lane_points(leftLines, image.shape[0])
    
    points = []
    points. append(rightLanePoints)
    points. append(leftLanePoints)
    #Draw lanes
    draw_lines(image, np.array(points), thickness=5)
            
def separate_lines_by_slope(lines, center_x):
    #Serapates lines by slope of the line and relative position to the center
    rightLanes = []
    leftLanes = []
    for line in lines:
        for x1,y1,x2,y2 in line:
            slope = (float(y2)-y1)/(x2-x1)
            if slope >= 0 and x1 > center_x and x2 > center_x:
                leftLanes.append([x1,y1,x2,y2])
            elif slope < 0 and x1 < center_x and x2 < center_x:
                rightLanes.append([x1,y1,x2,y2])
    right = np.array(rightLanes)
    left = np.array(leftLanes)
    return right, left

def get_lane_points(lines, lane_start_coord_y):
    #Takes lines list and creates one line that fits them the best
    lineX = []
    lineY = []
    for i in range(0,len(lines)):
        x1,y1,x2,y2 = lines[i,:]
        lineX.append(x1)
        lineX.append(x2)

        lineY.append(y1)
        lineY.append(y2)
        
    #Get line coefficients
    coeffs = np.polyfit(lineX, lineY, 1)
    
    minY = min(lineY)
    maxY = lane_start_coord_y #Image Coords start at top right so bottom of the image has bigger Y than top of the image 
    points = []
    
    minX = (minY - coeffs[1])/coeffs[0]
    maxX = (maxY - coeffs[1])/coeffs[0]
    
    points.append(int(minX))
    points.append(int(minY))
    points.append(int(maxX))
    points.append(int(maxY))
    
    points = np.array(points)
    return points

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    #Returns iamge with Hough lines drawn on it
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    #draw_lines(line_img, lines)
    draw_lanes(line_img, lines)
    return line_img

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    #Combines initial image and image with lines drawn on it
    return cv2.addWeighted(initial_img, α, img, β, λ)

In [ ]:
def process_image(image):
    processed_image = color_treshold(image)
    processed_image = grayscale(processed_image)
    processed_image = gaussian_blur(processed_image, 5)
    processed_image = canny(processed_image, 50, 150)
    #lanes are roughly in the same position on every image and shape below seems to fit it just right
    vertices = np.array([[(40,image.shape[0]),(470, 320), (500, 320), (image.shape[1]-20,image.shape[0])]], dtype=np.int32)
    processed_image = region_of_interest(processed_image, vertices)
    processed_image = hough_lines(processed_image, 2, np.pi/180, 15, 3, 5)
    processed_image = weighted_img(processed_image, image)
    return processed_image


Process_image takes image from file as an arguments and returns fully processed image with lines on top of road lane as a result

Iterate over every test image, process it and save to test_images_result

Video processing

In [ ]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
def process_video(path):
    #Drawns lines on every frame of the image and then saves new video on the disk
    video_name = path.split('/')[-1]
    output_dir = 'test_videos_output/'
    video_clip = VideoFileClip(path)
    video = video_clip.fl_image(process_image)
    %time video.write_videofile(output_dir + video_name, audio=False)

In [ ]:
white_path = "test_videos/solidWhiteRight.mp4"
process_video(white_path)

In [ ]:
yellow_path = "test_videos/solidYellowLeft.mp4"
process_video(yellow_path)

Play video inline

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format("test_videos_output/solidYellowLeft.mp4"))

In [ ]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format("test_videos_output/solidWhiteRight.mp4"))